# Scraping de la liste des PRADA sur le site de la CADA

http://www.cada.fr/personnes-responsables,6059.html

Car le formulaire n'est pas très convivial...

In [54]:
import re
import sys
import pickle
from collections import defaultdict

import requests
from bs4 import BeautifulSoup

# Scrap le formulaire

In [67]:
url = 'http://www.cada.fr/personnes-responsables,6059.html'
response = requests.get(url)
assert response.status_code == 200
assert response.encoding == 'utf-8'
text = response.text

In [68]:
soup = BeautifulSoup(text, 'html.parser')

In [69]:
form_type = soup.find(id="type")
list_type = [option['value'] for option in form_type.find_all('option')]

In [70]:
form_dept = soup.find(id="dept")
list_dept = [option['value'] for option in form_dept.find_all('option')]

# Scrap toutes les combinaisons

In [86]:
dict_prada = {}

for type_admin in list_type:
    dict_prada[type_admin] = {}
    print('Scrapping category "{}"'.format(type_admin))
    
    for dept in list_dept:
        sys.stdout.write('.')
        sys.stdout.flush()
        
        data = {
            'id_article': '6059',
            'page': 'article',
            'formulaire_action': 'prada',
            'formulaire_action_args': 'u2Tfw6pP/715JbIUQ9MMpjPSoXmgRPgQIoIu6+OCRphktQbarUOE29+UgR9cPvRSSzHQJpMQwqaMmBWv+kY+FhObKxr+ec0=',
            'type': type_admin,
            'dept': dept,
            'valide': '',
            'nobot': '',
        }

        response = requests.post(url, data=data)
        assert response.status_code == 200
        assert response.encoding == 'utf-8'
        text = response.text
        
        soup = BeautifulSoup(text, 'html.parser')
        
        chapo = soup.find("p", { "class" : "chapo" }).get_text()
        if chapo == "Recherche infructueuse. Il y n'a pas de résultats pour ces critères de recherche":
            blocs = []
        else:
            nb_results = int(re.match('(\d+) résultats pour vos critères de recherche ', chapo).groups()[0])

            blocs = soup.findAll("div", { "class" : "block-article" })
            if len(blocs) != nb_results:
                if (
                    (type_admin == "Établissement public territorial" and 
                    dept == "" and 
                    len(blocs) == 612 and 
                    nb_results == 613)
                    or
                    (type_admin == "Mairie" and 
                    dept == "" and 
                    len(blocs) == 679 and 
                    nb_results == 680)
                        ):
                    print('Count mismatch (already known)')
                else:
                    raise ValueError('Count mismatch : {}, {}, {} vs {}'.format(type_admin, dept, len(blocs), nb_results))
        
        dict_prada[type_admin][dept] = blocs
        
    print('')
      

Scrapping category ""
...........................................................................................................
Scrapping category "Autorité administrative indépendante"
...........................................................................................................
Scrapping category "Autre organisme privé"
...........................................................................................................
Scrapping category "Conseil départemental"
...........................................................................................................
Scrapping category "Conseil régional"
...........................................................................................................
Scrapping category "Établissement public d'État"
...........................................................................................................
Scrapping category "Établissement public territorial"
.Count mismatch (already known)
.............

# Save the dict

In [95]:
dict_prada_str = {}

for type_admin in list_type:
    dict_prada_str[type_admin] = {}
    for dept in list_dept:
        blocs = dict_prada[type_admin][dept]
        dict_prada_str[type_admin][dept] = [str(prada) for prada in blocs]

In [97]:
with open('dict_prada_str.pickle', 'wb') as f:
    pickle.dump(dict_prada_str, f)

In [6]:
with open('dict_prada_str.pickle', 'rb') as f:
    dict_prada_str = pickle.load(f)

# Remove mutiple occurrences

In [56]:
dict_by_bloc = defaultdict(list)
for type_admin, tmp in dict_prada_str.items():
    for dept, blocs in tmp.items():
        for bloc in blocs:
            dict_by_bloc[bloc].append((type_admin, dept))

In [60]:
list_prada = []
for bloc, list_categories in dict_by_bloc.items():
    l_type_admin = [categories[0] for categories in list_categories]
    set_type_admin = set(l_type_admin) - set([''])    
    assert(len(set_type_admin) <= 1)
    if set_type_admin:
        type_admin = list(set_type_admin)[0]
    else:
        type_admin = ''

    l_dept = [categories[1] for categories in list_categories]
    set_dept = set(l_dept) - set([''])
    assert(len(set_dept) <= 1)
    if set_dept:
        dept = list(set_dept)[0]
    else:
        dept = ''

    list_prada.append([type_admin, dept, bloc])

In [74]:
n_prada = len(list_prada)
n_prada

1628

In [72]:
dict_prada_str_deduplicated = {}
for type_admin in list_type:
    dict_prada_str_deduplicated[type_admin] = {}
    for dept in list_dept:
        dict_prada_str_deduplicated[type_admin][dept] = []
    
for type_admin, dept, bloc in list_prada:
    dict_prada_str_deduplicated[type_admin][dept].append(bloc)

# Sauvegarde le dictionnaire dédupliqué

In [75]:
with open('dict_prada_str_deduplicated.pickle', 'wb') as f:
    pickle.dump(dict_prada_str_deduplicated, f)

In [76]:
with open('dict_prada_str_deduplicated.pickle', 'rb') as f:
    dict_prada_str_deduplicated = pickle.load(f)

# Output the results as html

In [79]:
html_beginning = '''
<!DOCTYPE html>
<html lang="fr">
  <head>
    <meta charset="utf-8">
    <title>Liste des PRADA</title>
    <style>
body {
  font: 75%/1.5 Arial, Helvetica, sans-serif;
  color: #535353;
  background: #f2f2f2;
}
h2.titre_type {
    margin-bottom: .5em;
    color: #23aee3;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
    font-size: 2.5em;
    line-height: 1.25;
}
h2.titre_dept {
    margin-bottom: .5em;
    color: #23aee3;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
    font-weight: normal;
    font-size: 2em;
    line-height: 1.25;
}
h2.h3 {
    font-size: 1.3333333333333333em;
    margin-bottom: .25em;
    color: #244668;
    font-weight: bold;
    font-family: "Trebuchet MS", Verdana, "Lucida Grande", Tahoma, Helvetica, sans-serif;
}
.prada {
    color: #23AEE3;
    font-size: 16px;
    margin-bottom: 0px;
}
    </style>
  </head>
  <body>
'''

html_end = '''
  </body>
</html>
'''

html_page = html_beginning

html_page += 'Ce fichier contient {} PRADA.'.format(n_prada)

for type_admin in list_type:
    html_page += '<h2 class="titre_type">{}</h2>'.format(type_admin or 'Sans catégorie')
    for dept in list_dept:
        html_page += '<h2 class="titre_dept">{}</h2>'.format(dept or 'Sans département')
        blocs = dict_prada_str_deduplicated[type_admin][dept]
        for bloc in blocs:
            html_page += bloc

html_page += html_end

In [80]:
with open('Liste_PRADA.html', 'w') as f:
    f.write(html_page)

# Cut the blocs

In [114]:
regex = '<div class="block-article"><h2 class="h3">(.*)</h2><p class="prada"><strong>(.*)</strong></p><p>(.*)</p></div>'

list_prada = []
for type_admin in list_type:
    for dept in list_dept:
        blocs = dict_prada_str[type_admin][dept]
        for bloc in blocs:
            administration, nom, autre = re.match(regex, bloc).groups()
            autre = '; '.join([chunk for chunk in re.split('\n|\r|<br>|<br/>|</br>', autre) if chunk])
            list_prada.append([type_admin, dept, administration, nom, autre])

# Save as CSV

In [115]:
joined_string = ''.join([''.join(prada) for prada in list_prada])

In [116]:
used_chars = ''.join(sorted(list(set(joined_string))))
used_chars

' "\'(),-./0123456789;ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\xa0«°»ÇÈÉËÏÜàâçèéêëîïôûüÿœ–’'

In [117]:
separator = '|'
assert separator not in used_chars
assert '\n' not in used_chars

In [118]:
csv_content = '\n'.join(separator.join(prada) for prada in list_prada)

In [119]:
with open('Liste_PRADA.csv', 'w') as f:
    f.write(csv_content)